In [1]:
import torch
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)

PyTorch Version: 2.6.0+cu118
CUDA Available: True
CUDA Version: 11.8


In [2]:
import torch
print(torch.version.cuda)

11.8


In [1]:
import tensorflow as tf
#import tensorflow_hub as hub
from transformers import Wav2Vec2ForSequenceClassification, Wav2Vec2Processor
import torch
import params as yamnet_params
import yamnet as yamnet_model
params = yamnet_params.Params()
yamnet_model = yamnet_model.yamnet_frames_model(params)
yamnet_model.load_weights('yamnet.h5')
print("sunni punda")
from transformers import Wav2Vec2Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
wav2vec2_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self", torch_dtype=torch.float16).to(device)

#wav2vec2_model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h-lv60-self", torch_dtype=torch.float16, attn_implementation="flash_attention_2").to(device)




c:\Users\prave\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm





sunni punda


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
#fine tuning wav2vec model
from datasets import Dataset
import librosa
import pandas as pd

# Define a function to load and preprocess the audio files
def load_audio(file_path):
    audio, _ = librosa.load(file_path, sr=16000)
    return audio

# Define your dataset class
def create_dataset(ontology_data, class_map):
    data = []
    for category in ontology_data:
        for url in category.get("positive_examples", []):
            # Extract start and end times from the URL
            start_time, end_time = extract_times(url)  # You'll need the function extract_times

            file_name = f"{category['name']}.wav"
            download_audio_from_url(url, start_time, end_time, file_name, "audio_directory")

            # Load the audio file
            audio_data = load_audio(os.path.join("audio_directory", file_name))

            # Map the class
            label = class_map[category['id']]  # Use your CSV for this mapping

            data.append({"audio": audio_data, "label": label})
    
    return Dataset.from_pandas(pd.DataFrame(data))  # Convert to Hugging Face dataset format


In [4]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",          # output directory
    evaluation_strategy="epoch",     # evaluation strategy
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    logging_dir="./logs",            # directory for storing logs
)

trainer = Trainer(
    model=wav2vec2_model,                         # the model to be fine-tuned
    args=training_args,                  # training arguments
    train_dataset=train_dataset,         # the training dataset
    eval_dataset=eval_dataset,           # evaluation dataset
)

trainer.train()


NameError: name 'train_dataset' is not defined

In [8]:
save_directory = "wav2vec2_model"  # Path where you want to save the model
wav2vec2_model.save_pretrained(save_directory)

In [ ]:
import librosa
import numpy as np

# Function to load and preprocess audio for YAMNet
def preprocess_audio_yamnet(audio_path):
    waveform, sample_rate = librosa.load(audio_path, sr=16000)  # 16 kHz
    return waveform

# Function to load and preprocess audio for Wav2Vec2
def preprocess_audio_wav2vec2(audio_path):
    waveform, sample_rate = librosa.load(audio_path, sr=16000)  # 16 kHz
    return waveform


In [ ]:
import librosa
import numpy as np

# Function to load and preprocess audio for YAMNet
def preprocess_audio_yamnet(audio_path):
    waveform, sample_rate = librosa.load(audio_path, sr=16000)  # 16 kHz
    return waveform

# Function to load and preprocess audio for Wav2Vec2
def preprocess_audio_wav2vec2(audio_path):
    waveform, sample_rate = librosa.load(audio_path, sr=16000)  # 16 kHz
    return waveform
